In [133]:
# Solution attempt 3: basic neural network
# Final submission: 2018-06-06
# Submission score: 0.684

# This is intended to be a basic neural network thrown at the problem, with
# about the same level of preprocessing as the v01 logistic regression model did.

In [ ]:
import numpy as np
#import keras
import pandas as pd
#import feather
import re
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [2]:
def has_NAs(pd_series):
    return pd_series.isnull().values.any()

# This is deleting stuff that it shouldn't
def string_col_to_onehot(df, col_name):
    dummy_cols = pd.get_dummies(df[col_name], drop_first = True, prefix = col_name)
    df.drop(columns = [col_name], inplace = True)
    return(pd.concat([df, dummy_cols], axis = 1))

In [3]:
train = pd.read_csv("./Data Files/application_train.csv")
test = pd.read_csv("./Data Files/application_test.csv")

In [4]:
target = train["TARGET"]
train.drop("TARGET", inplace = True, axis = 1)
train_rows = len(train)

In [5]:
all_data = pd.concat([train,test], ignore_index = True)

In [6]:
building_info_columns = [c for c in train.columns.tolist() if re.search("_AVG$|_MODE$|_MEDI$", c)]
other_columns_to_remove = ["EXT_SOURCE_1","EXT_SOURCE_3","OWN_CAR_AGE","NAME_TYPE_SUITE"]

all_data.drop(building_info_columns + other_columns_to_remove, inplace = True, axis = 1)

In [7]:
object_cols = [col for col in all_data.columns if all_data[col].dtype == "O"]
for oc in object_cols:
    all_data = string_col_to_onehot(all_data, oc)

In [13]:
train = all_data.iloc[:train_rows,:].copy()
test = all_data.iloc[train_rows:,:].copy()

In [36]:
train.dropna(inplace = True)
target = target[train.index]

In [15]:
for col in test.columns:
    if test[col].isnull().sum() > 0:
        if test[col].dtype == "O":
            test[col] = test[col].fillna("Unknown")
        else:
            test.loc[test[col].isnull(),col] = test[col].median(skipna = True)

SK_ID_CURR
CNT_CHILDREN
AMT_INCOME_TOTAL
AMT_CREDIT
AMT_ANNUITY
AMT_GOODS_PRICE
REGION_POPULATION_RELATIVE
DAYS_BIRTH
DAYS_EMPLOYED
DAYS_REGISTRATION
DAYS_ID_PUBLISH
FLAG_MOBIL
FLAG_EMP_PHONE
FLAG_WORK_PHONE
FLAG_CONT_MOBILE
FLAG_PHONE
FLAG_EMAIL
CNT_FAM_MEMBERS
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
HOUR_APPR_PROCESS_START
REG_REGION_NOT_LIVE_REGION
REG_REGION_NOT_WORK_REGION
LIVE_REGION_NOT_WORK_REGION
REG_CITY_NOT_LIVE_CITY
REG_CITY_NOT_WORK_CITY
LIVE_CITY_NOT_WORK_CITY
EXT_SOURCE_2
OBS_30_CNT_SOCIAL_CIRCLE
DEF_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE
FLAG_DOCUMENT_2
FLAG_DOCUMENT_3
FLAG_DOCUMENT_4
FLAG_DOCUMENT_5
FLAG_DOCUMENT_6
FLAG_DOCUMENT_7
FLAG_DOCUMENT_8
FLAG_DOCUMENT_9
FLAG_DOCUMENT_10
FLAG_DOCUMENT_11
FLAG_DOCUMENT_12
FLAG_DOCUMENT_13
FLAG_DOCUMENT_14
FLAG_DOCUMENT_15
FLAG_DOCUMENT_16
FLAG_DOCUMENT_17
FLAG_DOCUMENT_18
FLAG_DOCUMENT_19
FLAG_DOCUMENT_20
FLAG_DOCUMENT_21
AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BU

In [18]:
for col in test.columns:
    print(test[col].dtype)

int64
int64
float64
float64
float64
float64
float64
int64
int64
float64
int64
int64
int64
int64
int64
int64
int64
float64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
uint8
ui

In [97]:
train2 = train.copy()
train2 = pd.concat([train2, target], axis = 1)
train2_real = train2.loc[target == 1,:]
train2 = pd.concat([train2_real,train2,train2_real])
train2 = train2.sample(frac = 1).reset_index(drop = True)
target2 = train2["TARGET"]
train2.drop("TARGET", axis = 1, inplace = True)

In [98]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train2 = sc.fit_transform(train2)

In [107]:
split = int(len(train2)*0.9)
train2_train, train2_test, target2_train, target2_test = train2[:split], train2[split:], target2[:split], target2[split:]

In [175]:
classifier = Sequential()
classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', input_dim = train2.shape[1]))
classifier.add(Dropout(rate = 0.3))
classifier.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', input_dim = train2.shape[1]))
classifier.add(Dropout(rate = 0.3))
#classifier.add(Dense(units = 30, kernel_initializer = 'uniform', activation = 'relu', input_dim = train2.shape[1]))
#classifier.add(Dropout(rate = 0.3))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [176]:
classifier.fit(train2_train, target2_train, batch_size = 20000, epochs = 25)

Epoch 1/25
274744/274744 [==============================] - 1s 5us/step - loss: 0.2286 - acc: 0.7539
Epoch 2/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1682 - acc: 0.7992
Epoch 3/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1536 - acc: 0.7992
Epoch 4/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1506 - acc: 0.7992
Epoch 5/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1488 - acc: 0.7992
Epoch 6/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1478 - acc: 0.7992
Epoch 7/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1471 - acc: 0.7992
Epoch 8/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1465 - acc: 0.7992
Epoch 9/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1461 - acc: 0.7992
Epoch 10/25
274744/274744 [==============================] - 1s 3us/step - loss: 0.1458 - a

In [177]:
train2_test2 = sc.fit_transform(train2_test)
predictions = classifier.predict(train2_test2)
roc_auc_score(target2_test, predictions)

0.716446800947082

In [106]:
from sklearn.metrics import roc_auc_score

In [178]:
data_predictions = classifier.predict(sc.fit_transform(test))

In [179]:
submission = pd.DataFrame({"SK_ID_CURR":test["SK_ID_CURR"], "TARGET":np.reshape(data_predictions, newshape = (-1))})

In [180]:
submission.to_csv("v03_prections.csv", index = False)